# Market Basket Analysis

This notebook uses the apriori algorithm to perform market basket analysis.

In [1]:
import pandas as pd

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [2]:
data = pd.read_csv('../data/clean_data/products_clean.csv', compression='zip', parse_dates=True, index_col='date')

In [3]:
data.head()

,territory,ar_div,customer_num,sold_to_name,item_num,item_description,item_type,unit_of_measure,weight,standard_price,quantity,price,year,month
date,,,,,,,,,,,,,,
2017-01-01,EB,0,1000026,Monterey Foods Market,039675777771,Have'a Corn Chips,CHIPS,EACH,4 oz,1.90,48.0,76.80,2017,1
2017-01-01,SF,0,1007292,Madera Liquor,039675777771,Have'a Corn Chips,CHIPS,EACH,4 oz,1.90,12.0,19.20,2017,1
2017-01-01,MR,0,1000282,Mollie Stone #1-Sausalito,704108725215,Casa Linda Enchilada Sauce,SAUCE,EACH,24 oz,3.51,12.0,42.12,2017,1
2017-01-01,SF,0,1007262,KD's Grog & Grocery,070415101562,Benzels Pennystick Rods,CHIPS,EACH,12 oz,1.81,6.0,10.86,2017,1
2017-01-01,SF,0,1007262,KD's Grog & Grocery,039675777771,Have'a Corn Chips,CHIPS,EACH,4 oz,1.90,6.0,9.60,2017,1


In [7]:
# this function transforms the dataframe into a usuable form for the apriori algorithm
# loops through each store and gets the orders for each month

def create_item_lists(df):

    # instantiate an empty list
    orders = []
    
    # loops over all the months
    for i in range(1,13):

        # creates a dataframe for the month, only selecting the store and item description
        df_month = df.loc[f'{df["year"].values[0]}-{i}'][['sold_to_name', 'item_description']]

        # loops over each store to combine all the items from that month
        for store in set(df['sold_to_name']):
            df_month_store = df_month[df_month['sold_to_name'] == store]

            items = [item for item in df_month_store['item_description']]
            orders.append(items)

    return orders

In [9]:
data_list = create_item_lists(data)

In [10]:
# encodes the list of orders to True/False to be used in the algorithm
te = TransactionEncoder()
te_ary = te.fit_transform(data_list)
df = pd.DataFrame(te_ary, columns=te.columns_)
df.head()

,"18 Rabbits Cashew, Grain, Caca",18 Rabbits Cherry Dk Choc Alm,"18 Rabbits Cherry, Chia, Vanil",18 Rabbits Date Pecan Coconut,18 Rabbits Fig Cranb Hazelnut,18 Rabbits Gracious Granola,18 Rabbits Jr Caramel Apple,18 Rabbits Jr Choc Banana,18 Rabbits Jr Mango Strawb,5-Hour Energy - Berry,...,Wiley Wallaby Grn Apple - 10 o,Wiley Wallaby Grn Apple - 7 oz,Wiley Wallaby Huckleberry - 7,Wiley Wallaby Red - 10 oz,Wiley Wallaby Red - 7 oz,Wiley Wallaby Red 2 oz,Wiley Wallaby Red Beans - 7 oz,Wiley Wallaby Red-Tub,Wiley Wallaby Watermelon - 10,Yiragacheffe Fair Trade Organi
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
frequent_items = apriori(df, min_support=0.07, use_colnames=True)

The min_support had to be set very low to find any relationships between items

In [12]:
rules = association_rules(frequent_items, metric='lift', min_threshold=1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bariani EV Olive Oil - 16 oz),(Baker's Delight Crumpets),0.093709,0.268482,0.071336,0.761246,2.835364,0.046177,3.063892
1,(Baker's Delight Crumpets),(Bariani EV Olive Oil - 16 oz),0.268482,0.093709,0.071336,0.265700,2.835364,0.046177,1.234225
2,(Cafe Fanny Original),(Baker's Delight Crumpets),0.109598,0.268482,0.091764,0.837278,3.118558,0.062339,4.495508
3,(Baker's Delight Crumpets),(Cafe Fanny Original),0.268482,0.109598,0.091764,0.341787,3.118558,0.062339,1.352758
4,(Baker's Delight Crumpets),(Casa Linda Enchilada Sauce),0.268482,0.156291,0.137484,0.512077,3.276445,0.095523,1.729187
...,...,...,...,...,...,...,...,...,...
621,(Tahoe Salt & Vinegar - 5 oz),"(La Morenita Salt & Lime, Tahoe Sea Salt - 5 o...",0.168612,0.093385,0.078470,0.465385,4.983494,0.062724,1.695826
622,(La Morenita Original),"(Tahoe Salt & Vinegar - 5 oz, Tahoe Sea Salt -...",0.246757,0.079442,0.078470,0.318003,4.002939,0.058867,1.349797
623,(La Morenita Salt & Lime),"(Tahoe Salt & Vinegar - 5 oz, Tahoe Sea Salt -...",0.243191,0.082036,0.078470,0.322667,3.933217,0.058519,1.355261
624,(Tahoe Sea Salt - 5 oz),"(La Morenita Salt & Lime, Tahoe Salt & Vinegar...",0.195850,0.083982,0.078470,0.400662,4.770820,0.062022,1.528384


In [13]:
rules[ 
    (rules['lift'] >= 3) &
    (rules['confidence'] >= 0.5) 
]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(Cafe Fanny Original),(Baker's Delight Crumpets),0.109598,0.268482,0.091764,0.837278,3.118558,0.062339,4.495508
4,(Baker's Delight Crumpets),(Casa Linda Enchilada Sauce),0.268482,0.156291,0.137484,0.512077,3.276445,0.095523,1.729187
5,(Casa Linda Enchilada Sauce),(Baker's Delight Crumpets),0.156291,0.268482,0.137484,0.879668,3.276445,0.095523,6.079163
7,(Comforts Chinese Chicken),(Baker's Delight Crumpets),0.094682,0.268482,0.086900,0.917808,3.418503,0.061480,8.900130
15,(Longitude Enchilada Sauce),(Baker's Delight Crumpets),0.085603,0.268482,0.078145,0.912879,3.400143,0.055162,8.396549
...,...,...,...,...,...,...,...,...,...
616,"(Tahoe Sea Salt - 5 oz, La Morenita Original)","(Tahoe Salt & Vinegar - 5 oz, Tahoe Bar-B-Que ...",0.126783,0.085603,0.078470,0.618926,7.230179,0.067616,2.399525
617,"(Tahoe Bar-B-Que - 5 oz, La Morenita Original)","(Tahoe Salt & Vinegar - 5 oz, Tahoe Sea Salt -...",0.106031,0.096952,0.078470,0.740061,7.633273,0.068190,3.474079
618,"(Tahoe Sea Salt - 5 oz, La Morenita Salt & Lime)","(Tahoe Salt & Vinegar - 5 oz, Tahoe Bar-B-Que ...",0.122892,0.087873,0.078470,0.638522,7.266432,0.067671,2.523330
619,"(Tahoe Bar-B-Que - 5 oz, La Morenita Salt & Lime)","(Tahoe Salt & Vinegar - 5 oz, Tahoe Sea Salt -...",0.104410,0.100843,0.078470,0.751553,7.452697,0.067941,3.619107


Nothing significant was found from doing market basket analysis. I will look into other options.